## SLIP Radiology Report Grading Welcome

Welcome to the SLIP project. The project was started with the intention to curate a set of clinically acquired Scans with Limited Imaging Pathology. The ongoing retrospective data collection process begins with checking the radiology reports (semi-structured written summaries of the contents of a scan). Because these scans are clinically acquired, a scan with no pathology cannot be marked as "healthy". Rather, scans are often marked as "unremarkable" or "appearing normal" to indicate a lack of finding. 

The training for SLIP report grading has 3 steps:
1. Reading SLIP and non-SLIP report examples
2. Practicing rating a subset of 150 reports
3. Grade a set of "shared" reports used for calculating reliability ratings

## Grading Guidelines

The grades are on an ordinal scale of 0/1/2 where
- Rating 0: Suspect serious imaging pathology
- Rating 1: Neither a 0 nor a 2 (AKA “I don’t know”). These are people who may or may not be included in our analyses depending on the context.
- Rating 2: No reason to suspect imaging pathology. These are people whose imaging pathology would likely not exclude them from being part of a control cohort in research brain MRI study. Note that this is different from their PAST MEDICAL HISTORY potentially excluding them, which is a different issue.

**Before starting to examine reports, read through the report grading guidelines
[HERE](https://docs.google.com/document/d/1-OIoUUyUxcYYEYNmO-H6XBuhudU2lqONuRGl8PR_HXk/edit?usp=sharing).** If you would like to add a phrase, please leave a comment in the document in the corresponding section containing the phrase and how it should be interpreted.

## Step 0: Run the following cell to load the necessary libraries and set up variables.

Then scroll down to Step 1.

In [ ]:
from google.cloud import bigquery
import pandas as pd
from annotationHelperLib import *
from reportMarkingFunctions import *
from IPython.display import clear_output

# This variable is a list of words that will be highlighed in yellow in the reports. 
# To add a specific string to the list, enclose it in single or double quotes and follow it
# with a comma
# wordsToHighlight = ['oma', 'surg', 'pongiform', 'NF1', 'mass', 'persistent', 'unchanged', 'stable', 'lobe', 'prolongation', 'intracranial']
highlightYellow = ['CLINICAL', 'INDICATION', 'HISTORY', 'REASON', 'IMPRESSION', 'FINDINGS', 'BRAIN',
                   'limited', 'MRA', 'MRI', 'ORBIT']
highlightGray = ['The ventricles and sulci are normal in size shape and configuration',
                 'There are no areas of abnormal enhancement following contrast administration',
                 'The visualized major intracranial vessels appear patent',
                 'No intra or extraaxial fluid collections are identified',
                 'No extra-axial fluid collections are identified',
                 'There are no areas of abnormal parenchymal signal intensity',
                 'No gross focal parenchymal signal abnormality is seen',
                 'No definite abnormality is seen in the visualized portions of the orbits',
                 'There are no areas of abnormal parenchymal signal intensity on these sequences',
                 'There are no areas of abnormal parechymal signal intensity',
                 'There is no parenchymal lesion or abnormal fluid collection',
                 'There is no parenchymal lesion or abnormal extra-axial collection',
                 'There is no mass effect or midline shift',
                 'There is no mass effect midline shift',
                 'No mass mass effect or abnormal enhancement is ',
                 'There is no mass or mass effect seen',
                 'No areas of hemorrhage are identified', 
                 'There is no abnormal intra- or extra-axial fluid collections',
                 'There is no evidence of intracranial hemorrhage or infarct',
                 'The ventricles and sulci are normal in size and configuration', 
                 'No parenchymal lesions or abnormal extra-axial collections are identified',
                 'There is no signal abnormality identified in the visualized parenchyma',
                 'There is no midline shift mass effect abnormal enhancement or acute infarction',
                 'There is no evidence of mass mass effect or midline shift',
                 'There is no intracranial mass lesion mass effect or midline shift',
                 'No intracranial hemorrhage midline shift or mass effect is appreciated',
                 'There are no mass effect midline shift abnormal enhancement or acute infarction',
                 'No mass effect midline shift or acute infarction is present',
                 'There are no intracranial mass lesions mass effect or midline shift',
                 'There is no intracranial mass effect or midline shift',
                 'There is no mass effect midline shift abnormal enhancement or acute infarction',
                 'No mass effect midline shift acute infarction or intracranial hemorrhage is identified',
                 'No mass effect midline shift abnormal enhancement or acute infarction is present',
                 'There is no brain parenchymal signal abnormality mass effect pathologic enhancement or restricted diffusion',
                 'There are no brain mass lesions mass effect or midline shift',
                 'There are no intracranial mass lesions',
                 'There is no mass effect midline shift or acute infarction',
                 'There are no areas of signal abnormality or pathologic enhancement', 
                 'No extra-axial collection mass effect or shift',
                 'no mass effect or midline shift',
                 'No intracranial mass lesion mass effect or midline shift is noted', 
                 'There are no intra- or extra-axial fluid collections',
                 'No abnormal fluid collections are identified',
                 'No extra-axial fluid collections are seen',
                 'No intra- or extra-axial fluid collections are identified',
                 'No abnormal extra-axial fluid collections are identified',
                 'No extraaxial fluid collections are identified',
                 'The midline structures are within normal limits',
                 'There is no restricted diffusion',
                 'The ventricular system is normal in size shape and configuration',
                 'The cerebellar tonsils are above the level of the foramen magnum',
                 'The cerebellar tonsils are at the level of foramen magnum',
                 'The cerebellar tonsils are at the level of the foramen magnum',
                 'The cerebellar tonsils do not project below the foramen magnum',
                 'The cerebellar tonsils are present at the foramen magnum',
                 'The cerebellar tonsils are above the foramen magnum',
                 'The cerebellar tonsils terminate above the level of the foramen magnum',
                 'The cerebellar tonsils terminate at the level of the foramen magnum',
                 'The cerebellar tonsils terminate at the foramen magnum', 
                 'The cerebellar tonsils are seen at the craniocervical junction',
                 'erebellar tonsils terminate above the foramen magnum',
                 'The cerebellar tonsils are at the foramen magnum',
                 'The myelination pattern is normal.',
                 'No brain volume loss.', 
                 'Ventricular size and configuration are normal.',
                 'Basilar cisterns remain patent.',
                 'No extra-axial fluid collection or mass effect on the brain',
                 'o abnormal parenchymal or leptomeningeal enhancement',
                 'The cerebellar tonsils are appropriately positioned',
                 'yelination is appropriate for age',
                 'he myelination pattern is appropriate',
                 'Myelination is within normal limits for age',
                 'The myelination is within range for age',
                 'Myelination appears appropriate for age',
                 'The major intracranial vessels appear patent',
                 'The hippocampi are symmetric and normal in size signal and internal architecture',
                 'No parenchymal lesions or abnormal extraaxial collections are identified', 
                 'No parenchymal lesions or abnormal extra-axial collections are identified',
                 'There is no mass effect midline shift abnormal enhancement or acute infarction',
                 'There are no areas of abnormal enhancement following gadolinium administration',
                 'The visualized major intracranial flow voids are present',
                 'There is mild mucosal thickening',
                 'mucus retention cyst',
                 'mucous retention cyst',
                 'o midline shift mass effect or acute infarction',
                 'The major intracranial arterial structures demonstrate normal flow-related enhancement',
                 'The major intracranial flow voids are unremarkable', 
                 'The visualized portions of the orbits are grossly unremarkable',
                 'There is no appreciable alteration in course or caliber', 
                 'No areas of aneurysmal outpouching are appreciated',
                 'No parenchymal lesion or abnormal fluid collection is identified',
                 'The hippocampi are symmetric in size and of normal signal intensity',
                 'The middle ear cavities mastoid air cells and paranasal sinuses are clear',
                 'The visualized portions of the orbits and mastoid air cells are grossly unremarkable',
                 'It should be noted that MRA may be insensitive for small (<3-4mm) aneurysms',
                 'There are no intraparenchymal signal abnormalities seen', 
                 'The ventricles are symmetric and normal in size', 
                 'No focal lesions are identified',
                 'parenchyma demonstrates no signal abnormalities',
                 'The brain appears normal in morphology',
                 'The posterior fossa structures are normal', 
                 'No intra-axial fluid collections are identified',
                 'The major intracranial vessels demonstrate normal flow',
                 'There is no midline shift or mass effect',
                 'without associated mass effect', 
                 'without significant mass effect',
                 'o areas of abnormal enhancement following gadolinium administration',
                 'he middle ear cavities and mastoid air cells are clear',
                 'neurohypophysis is orthotopic', 'Mild inferior cerebellar ectopia',
                 'There is no significant mass effect or midline shift',
                 'normal adenohypophysis', 'orthotopic neurohypophysis',
                 'here is no abnormal parenchymal enhancement on postcontrast images', 
                 'The ventricular system is normal in size and configuration', 
                 'There is no intracranial mass',
                 'Midline brain structures have normal morphology',
                 'the gyral/sulcal pattern is within normal limits.', 
                 'The myelination pattern is normal for age.',
                 'There are no areas of abnormal enhancement in the brain following contrast administration',
                 'Flow voids for the visualized major intracranial vessels appear patent',
                 'The paranasal sinuses are clear', 'The ventricular system is normal in size and shape',
                 'No definite areas of abnormal parenchymal signal',
                 'The visualized flow-voids of the major intracranial vessels appear patent',
                'cerebellar tonsils are rounded appearing and at the level of the foramen magnum']
highlightRed = ['artifact', 'rthodontic', 'istory of', 'motion', 'mass', 'cyst',
                'hypoplasia', 'signal abnormality', 'significant signal dropout',
                'ental hardware', 'leukemia', 'AML', 'ALL', 'CART therapy', 'CAR-T cell therapy',
                'Neurofibromatosis type 1', 'Chiari II',
                'multiple sclerosis', 'tumor resection', 'chemo',
                'thinning', 'thinned', 'bowing', 'bowed',
                'hyperintens', 'prolongation', 'spongiform dysplasia', 'nfarct',
                'This is an outside film. See images in PACS.',
                'volume loss', 'agenesis of the corpus callosum', 'neuroblastoma', 'NF1', 'NF 1',
                'mass effect', 'hypoxic ischemic encephalopathy',
                'This is a non-reportable study used for image storage. It has been automatically finalized and does not contain a result.']
highlightGreen = ['nremarkable brain', 'ormal brain',
                  'nremarkable', 'variant', 'variation', 
                  'normal anatomic variance', 'ncidental',
                  'within normal limit', 'developmental in etiology',
                  'doubtful significance', 'doubtful clinical significance',
                  'o brain abnormality', 'nremarkable mri of the brain',
                  'nremarkable mri of brain', 'not meeting criteria for Chiari I malformation',
                  'No evidence of intracranial MASS lesion hydrocephalus or intracranial hemorrhage',
                  'nremarkable noncontrast brain mri', 'nremarkable noncontrast brain MR',
                  'no clinical significance', 'no significance', 'nremarkable unenhanced brain',
                  'without signal abnormality', 'o significant intracranial abnormality',
                  'o intracranial abnormality',
                  'Normal MRI appearance of the brain',
                  'o structural abnormality', 'o evidence of intracranial abnormality']
phrasesIndicatingContrast = ['MRI WITHOUT AND WITH CONTRAST', 'following intravenous contrast',
                            'after the injection of gadolinium', 'post gadolinium injection']

toHighlight = {'gray': highlightGray, 
               'green': highlightGreen,
               'yellow': highlightYellow,
               'red': highlightRed }

## Step 1: reading SLIP and non-SLIP example reports

Run the following cell to view each sample report and its categorization as SLIP or non-SLIP. Press 'enter' after reading the report to view another report. When the cell finishes running, you can rerun it to view the reports again or proceed to the next section if you are comfortable with your understanding of the reports.

The argument `toHighlight` passes a dictionary with color strings as keys and lists of strings as values to the function. The function will highlight occurrences of strings in each list in the specified color. (Note: you can modify the lists included in this dictionary as you wish. The highlighting is imperfect, but it helps draw your eye to phrases that are meaningful to you.)

In [ ]:
readSampleReports(toHighlight)

## Step 2: Practice rating reports

In this section, you will practice rating reports yourself. Modify the variable `name` in the next cell to the name you use for publications. Then run the following cell to display a report, give it a grade, and note the reason you think it should get that grade. After your reasoning is submitted, the grades and reasons for the grades others have submitted will be shown. 

*If the report disappears after you type a number for the grade, use Ctrl + z or Cmd + z to undo, interrupt the kernel using the square button in the toolbar above, and run the cell again.*

In [ ]:
# Replace with the name you want used in publications of this data set
name = "Kurt Enernie"

isOnMainTable = welcomeUser(name)

In [ ]:
markSelfEvalReportSQL(name, toHighlight)

The following cell has the same functionality as the previous cell wrapped in a for loop so you can grade several reports while running the cell once. To change the number of reports in a batch, modify the variable `batchSize`.

In [ ]:
batchSize = 10

for i in range(batchSize):
    clear_output()
    print(i)
    print()
    if not isOnMainTable:
        markSelfEvalReportSQL(name, toHighlight)


## Next steps:

Now you can switch to the Arcus_Radiology_Report_Grading.ipynb notebook for the reliability reports and subsequent report grading.